In [1]:
!pip install sparknlp pymongo

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.jars.packages', 
         "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,com.databricks:spark-xml_2.12:0.18.0,com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3")
conf.set('spark.driver.memory','8g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)
spark

Ivy Default Cache set to: /home/ob2205/.ivy2/cache
The jars for the packages stored in: /home/ob2205/.ivy2/jars


:: loading settings :: url = jar:file:/ext3/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
com.databricks#spark-xml_2.12 added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-726dfd26-befe-4e89-ac34-3707f293b661;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s

In [7]:
PATH = '/scratch/ob2205/shared/ip/ip_address.txt'
ip_df = spark.read.text(PATH)
ip_address = ip_df.head(1)[0].value
ip_address

'10.32.32.6'

In [4]:
spark.sparkSession

In [5]:
from pymongo import MongoClient       

client = MongoClient(ip_address)

In [6]:
client.server_info()

ServerSelectionTimeoutError: 10.32.32.6:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66218f2dbc3f9a52d21670b5, topology_type: Unknown, servers: [<ServerDescription ('10.32.32.6', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('10.32.32.6:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
db=client.test
db

In [ ]:
# Get Scala Version
sc._gateway.jvm.scala.util.Properties.versionString()

In [ ]:
import os
from zipfile import ZipFile
source_path = "/scratch/work/public/proquest/proquest_hnp/BostonGlobe/BG_20151210212722_00001.zip"
with ZipFile(source_path, "r") as zip:
    print(zip.infolist())
    zip.extract('250949924.xml', 'zip_tmp')

In [ ]:
df = spark.read\
    .option('rootTag', 'Record')\
    .option('rowTag', 'Record')\
    .format("xml").load("zip_tmp/250949924.xml")

In [ ]:
df.printSchema()

In [ ]:
df.select('RecordID', 'Publisher', 'AlphaPubDate', 'NumericPubDate', 'RecordTitle', 'SourceType').show(2)

In [ ]:
# Save to mongo
df.write.format('com.mongodb.spark.sql.DefaultSource')\
    .option('uri',f'mongodb://{ip_address}/test.newspapers')\
    .save()

In [ ]:
db.list_collection_names()

In [ ]:
db.newspapers.find_one()

In [ ]:
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

In [ ]:
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)